In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from darts.models import NBEATSModel

In [4]:
import dask.dataframe as dd

In [5]:
from darts.timeseries import TimeSeries

##### Global Variables ####

In [6]:
DATA_DIR = '../data/'

In [7]:
input_chunk_length = 30
output_chunk_length = 1

In [8]:
features = [f'f_{i}' for i in range(300)]

#### Read Data #####

In [9]:
from IPython.display import clear_output
clear_output()

In [10]:
%%time 
train = dd.read_csv(DATA_DIR+'train.csv')

CPU times: user 187 ms, sys: 0 ns, total: 187 ms
Wall time: 187 ms


In [11]:
%%time
train_df = train.compute()

CPU times: user 11min 54s, sys: 4min 12s, total: 16min 7s
Wall time: 4min 29s


In [12]:
train_df.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624


In [13]:
# train_df = train_df.sample(frac=0.2)

In [14]:
investment_ids = train_df.investment_id.unique().tolist()

In [33]:
grp_df = train_df.groupby('investment_id',as_index=False).agg(c = ('target', len))

In [36]:
not_to_include_investment_ids = grp_df[grp_df.c < input_chunk_length+1].investment_id.unique().tolist()

In [15]:
train_df.shape

(3141410, 304)

In [16]:
#### past covariates will be features #####

##### Target and past covariates for each timeseries ######

In [37]:
data_dict = {}

In [39]:
def create_data_dict():
    for investment_id in investment_ids:
        if investment_id not in not_to_include_investment_ids:
            df = train_df[train_df.investment_id == investment_id]

            data_dict[str(investment_id)] = {}
            data_dict[str(investment_id)]['targets'] = TimeSeries.from_dataframe(df,time_col = 'time_id', value_cols = 'target')
            data_dict[str(investment_id)]['past_covariates'] = TimeSeries.from_dataframe(df,time_col = 'time_id', value_cols = features)
        
    return

In [40]:
%%time
create_data_dict()

CPU times: user 39.9 s, sys: 4.23 s, total: 44.1 s
Wall time: 44.1 s


### Model ####

In [49]:
model = NBEATSModel(input_chunk_length=input_chunk_length, 
                             output_chunk_length=output_chunk_length)

In [50]:
train_series = [data_dict[k]['targets'] for k in data_dict.keys()]

In [51]:
train_past_covariates_series = [data_dict[k]['past_covariates'] for k in data_dict.keys()]

In [52]:
model.fit(train_series[:100],past_covariates=train_past_covariates_series[:100], 
                 epochs=20, 
                 verbose=True)

[2022-01-26 23:05:18,222] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 117600 samples.
[2022-01-26 23:05:18,222] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 117600 samples.
[2022-01-26 23:05:18,837] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-01-26 23:05:18,837] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.


  0%|          | 0/20 [00:00<?, ?it/s]